In [1]:
import json
data_path = f"../../result/multitask_document_related.json"
with open(data_path, "r") as f:
    dataset = json.load(f)

In [ ]:
len(dataset)

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/upstage/llama-30b-instruct-2048" # "upstage/llama-30b-instruct-2048" # 589 ms
model_path = "AIDC-ai-business/Luban-13B" # "JoSw-14/LoKuS-13B"
tokenizer = AutoTokenizer.from_pretrained(model_path) 
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     torch_dtype=torch.float16
# )

/home/jovyan/workspace-main/venv_main/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
save_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/AIDC-ai-business/Luban-13B"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/AIDC-ai-business/Luban-13B/tokenizer_config.json',
 '/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/AIDC-ai-business/Luban-13B/special_tokens_map.json',
 '/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/AIDC-ai-business/Luban-13B/tokenizer.json')

In [6]:
%%time
max_len = 0
for ind, data in dataset.items():
    label = ", ".join(data['label'])
    inputs = tokenizer(label, return_tensors="pt")
    
    input_len = inputs['input_ids'].shape[1]
    if input_len > max_len:
        max_len = input_len
print(max_len)

50
CPU times: user 245 ms, sys: 3.13 ms, total: 249 ms
Wall time: 245 ms


In [12]:
# upstage/llama-30b-instruct-2048
# JoSw-14/LoKuS-13B
import time
for ind, data in dataset.items():
    if int(ind) > 10:
        st = time.time()
        query = data['query']
        pred_facet_list = data['pred']
        pred_facets = ", ".join(pred_facet_list)

        one_shot = """### User:\nThe predicted facets for 'caesars atlantic city' are 'parking, hotels'. But the correct facets are 'caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking'\n"""
        two_shot = """The predicted facets for 'vista, ca' are 'parking, hotels'. But the correct facets are 'weather, zip code, population, homes for sale'\n\n"""
        # prompt = one_shot + two_shot + f"""As in the example above, modify the predicted facets.\nThe predicted facets for '{query}' are '{pred_facets}'. What are the correct facets?\n\n### Assistant:\n"""
        prompt = one_shot + two_shot + f"""As in the example above, modify the predicted facets.\nThe predicted facets for '{query}' are '{pred_facets}'. What are the correct facets?\n\n### Assistant:\nThe correct facets for '{query}' are"""
        print(prompt)
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        label_str = ", ".join(data['label'])
        label_inputs = tokenizer(label_str, return_tensors="pt")
        label_len = label_inputs['input_ids'].shape[1]

        output = model.generate(**inputs, use_cache=True, max_new_tokens=int(label_len*2), temperature=0, top_p=1)
        
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        correct_facets = output[len(prompt):]
        correct_facet_list = [x.strip() for x in correct_facets.strip().split("\n")[0].strip("'").strip(".").strip("'").split(",") if x.strip() != ""]
        print(correct_facets)
        print(correct_facet_list)
        
        print("### Label:")
        print(data['label'])
        ed = time.time()
        print(ed-st)
        break

### User:
The predicted facets for 'caesars atlantic city' are 'parking, hotels'. But the correct facets are 'caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking'
The predicted facets for 'vista, ca' are 'parking, hotels'. But the correct facets are 'weather, zip code, population, homes for sale'

As in the example above, modify the predicted facets.
The predicted facets for 'device manager' are 'device manager, windows device manager'. What are the correct facets?

### Assistant:
The correct facets for 'device manager' are
 'device manager, device manager windows 10, device manager windows 7
['device manager', 'device manager windows 10', 'device manager windows 7']
### Label:
['open device manager', 'use device manager']
0.6051514148712158


In [16]:
correct_facets

" 'device manager, device manager windows 10, device manager windows 7"

In [ ]:
len(dataset)*98/60/60

In [ ]:
import re
eng_rule = re.compile('\'[a-zA-Z0-9,\s]+\'')
correct_facets = """
 'download, installation, compatibility, updates'."""
correct_facets = """
 'suva beauty products, suva beauty salon, suva beauty reviews'"""
print(correct_facets)
correct_facet_list = [x.strip() for x in correct_facets.strip().strip("'").strip(".").strip("'").split(",") if x.strip() != ""]
correct_facet_list

In [ ]:
for ind, data in dataset.items():
    if int(ind) == 69:
        query = data['query']
        pred_facet_list = data['pred']
        pred_facets = ", ".join(pred_facet_list)

        one_shot = """### User:\nThe facets for 'caesars atlantic city' are 'caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking'\n"""
        two_shot = """The facets for 'vista, ca' are 'weather, zip code, population, homes for sale'\n\n"""
        prompt = one_shot + two_shot + f"""### Assistant:\nThe correct facets for '{query}' are"""
        # prompt = one_shot + two_shot + f"""As in the examples above, create facets.\nWhat facets correspond to the {query}?\n\n### Assistant:\nThe correct facets for '{query}' are"""
        print(prompt)
        
        label_str = ", ".join(data['label'])
        label_inputs = tokenizer(label_str, return_tensors="pt")
        label_len = label_inputs['input_ids'].shape[1]
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        output = model.generate(**inputs, use_cache=True, max_new_tokens=int(label_len*1.5), temperature=0)
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        correct_facets = output[len(prompt):]
        print(correct_facets)
        
        print("### Label:")
        print(data['label'])
        break

## 에러 확인하기

In [ ]:
import re
eng_rule = re.compile('\'[a-zA-Z0-9\-&.,\s]+\'')
correct_facets = "The correct facets for 'fps' are 'fps windows 10, fps windows 7, fps xbox one, fps ps4'."
parsing = eng_rule.findall(correct_facets)
print(parsing)
correct_facet_list = [x.strip() for x in parsing[1].strip("'").split(",")]

In [ ]:
import json
data_path = f"../../result/LLM_multitask_document_related.json"
with open(data_path, "r") as f:
    dataset = json.load(f)
    
# data_path = f"../../result/LLM_multitask_related_error.json"
# with open(data_path, "r") as f:
#     dataset_error = json.load(f)

In [ ]:
len(dataset), len(dataset_error)

In [ ]:
final_dataset = {}
error_list = []
for ind, data in dataset.items():
    final_dataset[ind] = data
    
for ind, data in dataset_error.items():
    query = data['query']
    pred = data['pred']
    filter_pred = pred.replace(f"'{query}'", '')
    correct_facets = eng_rule.findall(filter_pred)
    if len(correct_facets) == 1:
        correct_facet_list = [x.strip() for x in correct_facets[0].strip("'").split(",")]

        final_dataset[ind] = {}
        final_dataset[ind]['query'] = data['query']
        final_dataset[ind]['pred'] = correct_facet_list
        final_dataset[ind]['label'] = data['label']
        final_dataset[ind]['options_overall_label'] = data['options_overall_label']
    else:
        # print(pred)
        # print(filter_pred)
        # print(correct_facets)
        error_list.append([pred, correct_facets, data['label']])

In [ ]:
final_dataset = {}
for ind, data in dataset.items():
    final_dataset[ind] = {}
    final_dataset[ind]['query'] = data['query']
    final_dataset[ind]['pred'] = [x.strip("'") for x in data['pred'] if x.strip()!=""]
    final_dataset[ind]['label'] = data['label']
    final_dataset[ind]['options_overall_label'] = data['options_overall_label']

In [ ]:
final_dataset['7']

In [ ]:
save_path = f"../../result/LLM_multitask_document_related.json"
with open(save_path, "w", encoding='utf-8') as f:
    json.dump(final_dataset, f)

In [ ]:
save_path = f"../../result/LLM_multitask_document_related1.json"
with open(save_path, "w", encoding='utf-8') as f:
    json.dump(dataset, f)